In [1]:
import numpy as np
import torch
from upcycle.logging import DataFrameLogger as logger
import pickle
import os
import yaml
import tqdm
import gnosis


from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Models

In [2]:
from gnosis.models import PreResNet
from gnosis.models import ClassifierEnsemble

In [3]:
def make_resnet(depth):
    return PreResNet(name="preresnet", num_classes=100, depth=depth, input_size=32, skip_connections=True)

## Datasets

In [4]:
from torchvision.datasets import CIFAR100
import torchvision

In [5]:
def get_unit_cube_normalization():
    min_vals = [0., 0., 0.]
    max_vals = [1., 1., 1.]
    offset = [0.5 * (min_val + max_val) for min_val, max_val in zip(min_vals, max_vals)]
    scale = [(max_val - min_val) / 2 for max_val, min_val in zip(max_vals, min_vals)]
    return torchvision.transforms.Normalize(offset, scale)


def make_cifar100(transforms, tensor_transforms):
    normalize_transforms = [
        torchvision.transforms.ToTensor(),
        get_unit_cube_normalization()
    ]
    transforms = torchvision.transforms.Compose(transforms + normalize_transforms + tensor_transforms)
    
    train_dataset = CIFAR100(root="/datasets", download=True, train=True, transform=transforms)
    test_dataset = CIFAR100(root="/datasets", download=True, train=False, transform=transforms)
    
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1000, shuffle=False)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)
    
    return train_loader, test_loader


clean_train_loader, clean_test_loader = make_cifar100([], [])

Files already downloaded and verified
Files already downloaded and verified


## Utils

In [6]:
import torch.nn.functional as F

In [7]:
# def mixup_data(x, alpha=1., seed):
#     np.random.seed(seed)
#     lam = np.random.beta(alpha, alpha)
#     batch_size = x.size()[0]
#     index = try_cuda(torch.randperm(batch_size))
#     mixed_x = lam * x + (1 - lam) * x[index, :]
#     return mixed_x

In [8]:
def compute_probs(model, loader, use_mixup=False):
    model.eval()
    probs_list, labels_list = [], []
    with torch.no_grad():
        for x, y in tqdm.tqdm(loader):
#             if use_mixup
            logits = model(x.cuda())
            probs = F.softmax(logits, dim=-1)
            probs_list.append(probs.detach().cpu().numpy())
            labels_list.append(y.detach().numpy())
    return np.concatenate(probs_list), np.concatenate(labels_list)

In [9]:
def get_accuracy(probs, labels):
    preds = np.argmax(probs, axis=-1)
    assert preds.shape == labels.shape
    return (preds == labels).mean()

In [10]:
torch.cuda.empty_cache()

## File traversal

In [27]:
def is_leaf_directory(path):
    return all([file in os.listdir(path) for file in ["student.ckpt"]])


def process_all_subdirectories(path, process_leaf_directory):
#     print(path)
    if not os.path.isdir(path):
        return 
    if is_leaf_directory(path):
        process_leaf_directory(path)
        return
    for subdir in os.listdir(path):
        full_path = os.path.join(path, subdir)
        process_all_subdirectories(full_path, process_leaf_directory)

In [28]:
def get_student_model(config, path):
    student_model = make_resnet(config['classifier']['depth']).cuda()
    print("Student:")
    print("\tDepth: {}".format(config['classifier']['depth']))
    with open(os.path.join(path, "student.ckpt"), "rb") as f:
        student_model.load_state_dict(pickle.load(f))
    return student_model.eval()

def get_teacher_model(config, path):
    teachers = []
    print("Teacher:")
    print("\tDepth: {}".format(config['teacher']['depth']))
    print("\tNumber of components: {}".format(config['teacher']['num_components']))
    
    for i in range(config["teacher"]["num_components"]):
        teacher = make_resnet(config['teacher']['depth'])
        teachers.append(teacher.cuda())
        
    if "teacher_ensemble.ckpt" in os.listdir(path):
        teacher_ensemble = ClassifierEnsemble(*teachers)
        with open(os.path.join(path, "teacher_ensemble.ckpt"), "rb") as f:
            teacher_ensemble.load_state_dict(pickle.load(f))
        return teacher_ensemble.eval()

    elif "teacher_0.ckpt" in os.listdir(path):
        for i in range(config["teacher"]["num_components"]):
            with open(os.path.join(path, "teacher_{}.ckpt".format(i)), "rb") as f:
                teachers[i].load_state_dict(pickle.load(f))
            teacher_ensemble = ClassifierEnsemble(*teachers)
        return teacher_ensemble
    else:
        print("No teacher checkpoints found")
        return None
    
def get_config(path):
    path1 = os.path.join(path, "hydra_config.txt")
    path2 = os.path.join(path, ".hydra/config.yaml")
    if os.path.exists(path1):
        path = path1
    else:
        path = path2
    with open(path, "rb") as f:
        config = yaml.full_load(f)
    return config

In [33]:
def make_const_aug_loader(aug_train_loader, aug_test_loader):
    def fn(config):
        return aug_train_loader, aug_test_loader
    return fn


def make_student_process_leaf_directory(
    train_loader, test_loader, get_aug_loaders,
    recompute_existing=False
):
    def process_leaf_directory(path):
        print("Pocessing {}".format(path))
        
        config = get_config(path)
        aug_train_loader, aug_test_loader = get_aug_loaders(config)
        
        print("Trial: {}".format(config["trial_id"]))

        student_model = get_student_model(config, path)
        teacher_model = get_teacher_model(config, path)
        
        models_files = [[student_model, "student_probs.npz"], 
                        [teacher_model, "teacher_probs.npz"]]
#         models_files = [[teacher_model, "teacher_probs.npz"]]
#         models_files = [[student_model, "student_probs.npz"]]

        if all([os.path.exists(os.path.join(path, file))
               for _, file in models_files]) and not recompute_existing:
            print("Files already exist, skipping...")
            return
        
        for model, file in models_files:
            if model is None:
                print("Warning: no model for {}".format(file))
                continue
            filename = os.path.join(path, file)
            train_probs, train_labels = compute_probs(model, train_loader)
            test_probs, test_labels = compute_probs(model, test_loader)
            aug_train_probs, _ = compute_probs(model, aug_train_loader)
            aug_test_probs, _ = compute_probs(model, aug_test_loader)
            print("Done...")
            np.savez(
                filename,
                train_probs=train_probs,
                test_probs=test_probs,
                aug_train_probs=aug_train_probs,
                aug_test_probs=aug_test_probs,
                train_labels=train_labels,
                test_labels=test_labels
                )
        
    return process_leaf_directory

## Baseline distillation

In [148]:
baseline_train_loader, baseline_test_loader = make_cifar100(
    [torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

process_leaf_directory_fn = make_student_process_leaf_directory(
    clean_train_loader, clean_test_loader, baseline_train_loader, baseline_test_loader, recompute_existing=True)

Files already downloaded and verified
Files already downloaded and verified


In [139]:
base_dir = "../../gnosis_checkpoints/pavel_baseline_distillation_soft_only_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/"
process_all_subdirectories(base_dir, process_leaf_directory_fn)

Pocessing ../../gnosis_checkpoints/pavel_baseline_distillation_soft_only_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_1/2021-01-24_10-59-18
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3


100%|██████████| 10/10 [00:03<00:00,  2.57it/s]


Done...
Pocessing ../../gnosis_checkpoints/pavel_baseline_distillation_soft_only_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_3/2021-01-24_10-59-18
Trial: 3
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3


100%|██████████| 10/10 [00:03<00:00,  2.61it/s]


Done...
Pocessing ../../gnosis_checkpoints/pavel_baseline_distillation_soft_only_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_2/2021-01-24_10-59-18
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3


...

Done...


  0%|          | 0/50 [00:00<?, ?it/s]

Pocessing ../../gnosis_checkpoints/pavel_baseline_distillation_soft_only_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet20_3-preresnet20_1/trial_2/2021-01-24_10-59-18
Trial: 2
Student:
	Depth: 20
Teacher:
	Depth: 20
	Number of components: 3


100%|██████████| 10/10 [00:02<00:00,  4.20it/s]


Done...


## Architecture Cross-Over

In [149]:
baseline_train_loader, baseline_test_loader = make_cifar100(
    [torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

process_leaf_directory_fn = make_student_process_leaf_directory(
    clean_train_loader, clean_test_loader, baseline_train_loader, baseline_test_loader, recompute_existing=True)

Files already downloaded and verified
Files already downloaded and verified


In [150]:
base_dir = "../../gnosis_checkpoints/architecture_crossover_300epochs/"
process_all_subdirectories(base_dir, process_leaf_directory_fn)

Pocessing ../../gnosis_checkpoints/architecture_crossover_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet110_3-preresnet56_1/trial_1/2021-01-24_18-15-54
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 110
	Number of components: 3


100%|██████████| 10/10 [00:02<00:00,  3.95it/s]


Done...
Pocessing ../../gnosis_checkpoints/architecture_crossover_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet110_3-preresnet56_1/trial_3/2021-01-24_18-15-54
Trial: 3
Student:
	Depth: 56
Teacher:
	Depth: 110
	Number of components: 3


100%|██████████| 10/10 [00:02<00:00,  3.94it/s]


Done...
Pocessing ../../gnosis_checkpoints/architecture_crossover_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet110_3-preresnet56_1/trial_2/2021-01-24_18-15-54
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 110
	Number of components: 3


...

Done...
Pocessing ../../gnosis_checkpoints/architecture_crossover_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet20_1-preresnet56_1/trial_3/2021-01-24_18-15-54
Trial: 3
Student:
	Depth: 56
Teacher:
	Depth: 20
	Number of components: 1


  0%|          | 0/50 [00:00<?, ?it/s]

Done...
Pocessing ../../gnosis_checkpoints/architecture_crossover_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet20_1-preresnet56_1/trial_2/2021-01-24_18-15-54
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 20
	Number of components: 1


100%|██████████| 10/10 [00:02<00:00,  3.94it/s]

Done...


## MixUP

In [152]:
baseline_train_loader, baseline_test_loader = make_cifar100(
    [torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

process_leaf_directory_fn = make_student_process_leaf_directory(
    clean_train_loader, clean_test_loader, baseline_train_loader, baseline_test_loader, recompute_existing=True)

Files already downloaded and verified
Files already downloaded and verified


In [153]:
base_dir = "../../gnosis_checkpoints/mixup_300epochs/"
process_all_subdirectories(base_dir, process_leaf_directory_fn)

Pocessing ../../gnosis_checkpoints/mixup_300epochs/cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet56_3-preresnet56_1/trial_1/2021-01-24_18-06-59
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:04<00:00,  2.48it/s]


Done...
Pocessing ../../gnosis_checkpoints/mixup_300epochs/cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet56_3-preresnet56_1/trial_3/2021-01-24_18-06-59
Trial: 3
Student:
	Depth: 56


...

Done...
Pocessing ../../gnosis_checkpoints/mixup_300epochs/cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet56_5-preresnet56_1/trial_2/2021-01-24_18-06-59
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 5


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:05<00:00,  1.90it/s]


Done...


## SVHN augmentation

In [16]:
from gnosis.datasets.svhn_augmentation import make_cifar100_svhn

In [17]:
transforms = [torchvision.transforms.RandomCrop(32, 4), 
              torchvision.transforms.RandomHorizontalFlip()]
normalize_transforms = [
        torchvision.transforms.ToTensor(),
        get_unit_cube_normalization()
    ]
transforms = torchvision.transforms.Compose(transforms + normalize_transforms)

In [18]:
train_dataset = make_cifar100_svhn(root="/datasets", download=True, num_svhn_data=20000, train=True, transform=transforms)
test_dataset = make_cifar100_svhn(root="/datasets", download=True, num_svhn_data=20000, train=False, transform=transforms)

cifar100svhn_train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1000, shuffle=False)
cifar100svhn_test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

Files already downloaded and verified
Using downloaded and verified file: /datasets/train_32x32.mat
Files already downloaded and verified


In [19]:
process_leaf_directory_fn = make_student_process_leaf_directory(
    clean_train_loader, clean_test_loader, cifar100svhn_train_loader, cifar100svhn_test_loader, recompute_existing=True)

In [20]:
base_dir = "../../gnosis_checkpoints/svhn_augmentation/"
process_all_subdirectories(base_dir, process_leaf_directory_fn)

Pocessing ../../gnosis_checkpoints/svhn_augmentation/synth_aug_0.0_cifar100_svhn_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_1/2021-01-22_22-59-20
Trial: 1


  0%|          | 0/50 [00:00<?, ?it/s]

Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


...

Done...
Pocessing ../../gnosis_checkpoints/svhn_augmentation/synth_aug_0.0_cifar100_svhn_ts_soft_cross_ent_v0.0.10/preresnet56_5-preresnet56_1/trial_2/2021-01-22_22-59-20
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 5


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


Done...


## Uniform inputs

In [26]:
import gnosis.utils.custom_augmentations

In [27]:
uniform_train_loader, uniform_test_loader = make_cifar100(
    [torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], 
    [gnosis.utils.custom_augmentations.MaybeReplaceWithUniform(p=0.2, min=-1., max=1.)])

process_leaf_directory_fn = make_student_process_leaf_directory(
    clean_train_loader, clean_test_loader, uniform_train_loader, uniform_test_loader, recompute_existing=True)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
base_dir = "../../gnosis_checkpoints/uniform_inputs//"
process_all_subdirectories(base_dir, process_leaf_directory_fn)

Pocessing ../../gnosis_checkpoints/uniform_inputs//synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/['replace_with_uniform']_0.2/preresnet56_3-preresnet56_1/trial_1/2021-01-24_02-20-16
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:04<00:00,  2.42it/s]


Done...
Pocessing ../../gnosis_checkpoints/uniform_inputs//synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/['replace_with_uniform']_0.2/preresnet56_3-preresnet56_1/trial_2/2021-01-24_02-20-16
Trial: 2
Student:
	Depth: 56


...

Done...
Pocessing ../../gnosis_checkpoints/uniform_inputs//synth_aug_0.0_cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/['replace_with_uniform']_0.2/preresnet56_5-preresnet56_1/trial_0/2021-01-24_02-20-16
Trial: 0
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 5


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:05<00:00,  1.88it/s]


Done...


## Augmentation Ablation

In [49]:
cj_train_loader, cj_test_loader = make_cifar100(
    [torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), 
     torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

vflip_train_loader, vflip_test_loader = make_cifar100(
    [torchvision.transforms.RandomVerticalFlip(), 
     torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

rot_train_loader, rot_test_loader = make_cifar100(
    [torchvision.transforms.RandomRotation(degrees=20), 
     torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

all_train_loader, all_test_loader = make_cifar100(
    [torchvision.transforms.RandomRotation(degrees=20),
     torchvision.transforms.RandomVerticalFlip(),
     torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
     torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

def get_aug_loaders(config):
    if config["augmentation"]["transforms_list"] == "colorjitter,crop,horizontal_flip":
        return cj_train_loader, cj_test_loader
    elif config["augmentation"]["transforms_list"] == "vertical_flip,crop,horizontal_flip":
        return vflip_train_loader, vflip_test_loader
    elif config["augmentation"]["transforms_list"] == "rotation,crop,horizontal_flip":
        return rot_train_loader, rot_test_loader
    elif config["augmentation"]["transforms_list"] == "rotation,vertical_flip,colorjitter,crop,horizontal_flip":
        return all_train_loader, all_test_loader
    else:
        print(config["augmentation"]["transforms_list"])

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [50]:
process_leaf_directory_fn = make_student_process_leaf_directory(
    clean_train_loader, clean_test_loader, get_aug_loaders, recompute_existing=True)

In [51]:
base_dir = "../../gnosis_checkpoints/aug_ablation/"
process_all_subdirectories(base_dir, process_leaf_directory_fn)

Pocessing ../../gnosis_checkpoints/aug_ablation/augmentations_colorjitter,crop,horizontal_flip_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_1/2021-01-22_14-42-46
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:07<00:00,  1.31it/s]


Done...
Pocessing ../../gnosis_checkpoints/aug_ablation/augmentations_colorjitter,crop,horizontal_flip_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_3/2021-01-22_14-42-46
Trial: 3
Student:
	Depth: 56


...

Done...
Pocessing ../../gnosis_checkpoints/aug_ablation/augmentations_rotation,crop,horizontal_flip_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_5-preresnet56_1/trial_2/2021-01-22_14-42-46
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 5


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:05<00:00,  1.77it/s]


Done...


## Temperature ablation

In [34]:
baseline_train_loader, baseline_test_loader = make_cifar100(
    [torchvision.transforms.RandomCrop(32, 4), 
     torchvision.transforms.RandomHorizontalFlip()], [])

process_leaf_directory_fn = make_student_process_leaf_directory(
    clean_train_loader, clean_test_loader, 
    make_const_aug_loader(baseline_train_loader, baseline_test_loader), 
    recompute_existing=False)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
base_dir = "../../gnosis_checkpoints/temp_ablation/temp_ablation/"
process_all_subdirectories(base_dir, process_leaf_directory_fn)

Pocessing ../../gnosis_checkpoints/temp_ablation/temp_ablation/cifar100_temp_4.0/preresnet56_3-preresnet56_1/trial_1/2021-01-26_21-39-42
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3
Files already exist, skipping...
Pocessing ../../gnosis_checkpoints/temp_ablation/temp_ablation/cifar100_temp_4.0/preresnet56_3-preresnet56_1/trial_2/2021-01-26_21-39-42
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3
Files already exist, skipping...
Pocessing ../../gnosis_checkpoints/temp_ablation/temp_ablation/cifar100_temp_4.0/preresnet56_3-preresnet56_1/trial_0/2021-01-26_21-39-42
Trial: 0
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3
Files already exist, skipping...
Pocessing ../../gnosis_checkpoints/temp_ablation/temp_ablation/cifar100_temp_4.0/preresnet56_1-preresnet56_1/trial_1/2021-01-26_21-39-42
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 1
Files already exist, skipping...
Pocessing ../../gnosis_c

  0%|          | 0/50 [00:00<?, ?it/s]

Files already exist, skipping...
Pocessing ../../gnosis_checkpoints/temp_ablation/temp_ablation/cifar100_temp_1.0/preresnet56_3-preresnet56_1/trial_1/2021-01-18_04-21-35
Trial: 1
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3
No teacher checkpoints found


  0%|          | 0/50 [00:00<?, ?it/s]

Done...
Pocessing ../../gnosis_checkpoints/temp_ablation/temp_ablation/cifar100_temp_1.0/preresnet56_3-preresnet56_1/trial_2/2021-01-18_04-21-35
Trial: 2
Student:
	Depth: 56
Teacher:
	Depth: 56
	Number of components: 3
No teacher checkpoints found


...

Done...
Pocessing ../../gnosis_checkpoints/temp_ablation/temp_ablation/cifar100_temp_1.0/preresnet56_1-preresnet110_1/trial_0/2021-01-22_23-51-58
Trial: 0
Student:
	Depth: 110
Teacher:
	Depth: 56
	Number of components: 1


  0%|          | 0/50 [00:00<?, ?it/s]

Done...


100%|██████████| 10/10 [00:02<00:00,  3.84it/s]

Done...


## Develop

In [133]:
teacher_probs = np.load("../../gnosis_checkpoints/pavel_baseline_distillation_soft_only_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet110_3-preresnet110_1/trial_1/2021-01-24_10-59-18/teacher_probs.npz")
student_probs = np.load("../../gnosis_checkpoints/pavel_baseline_distillation_soft_only_300epochs/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet110_3-preresnet110_1/trial_1/2021-01-24_10-59-18/student_probs.npz")

In [134]:
print(get_accuracy(teacher_probs["train_probs"], teacher_probs["train_labels"]))
print(get_accuracy(teacher_probs["test_probs"], teacher_probs["test_labels"]))
print(get_accuracy(teacher_probs["aug_train_probs"], teacher_probs["train_labels"]))
print(get_accuracy(teacher_probs["aug_test_probs"], teacher_probs["test_labels"]))
print()
print(get_accuracy(student_probs["train_probs"], student_probs["train_labels"]))
print(get_accuracy(student_probs["test_probs"], student_probs["test_labels"]))
print(get_accuracy(student_probs["aug_train_probs"], student_probs["train_labels"]))
print(get_accuracy(student_probs["aug_test_probs"], student_probs["test_labels"]))

0.9998
0.7394
0.9998
0.7377

0.9998
0.7394
0.9998
0.7368


In [27]:
path = "../../gnosis_checkpoints/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet110_1-preresnet110_1/trial_1/2021-01-21_13-43-40/"

In [31]:
with open(os.path.join(path, "hydra_config.txt"), "rb") as f: #".hydra/config.yaml"), "rb") as f:
    config = yaml.full_load(f)

In [32]:
config['teacher']

{'depth': 110,
 'num_components': 1,
 'use_ckpts': True,
 'shuffle_ckpts': False,
 'ckpt_dir': 'data/experiments/image_classification/classifier_checkpoints/preresnet110_cifar100_unitcube'}

In [ ]:
teacher_model = ClassifierEnsemble()

In [74]:
# def process_file(path):
# #     if path.split("/")[-1] == 'student.ckpt':
# #         print(path)
#     if path.split("/")[-1] == 'config.yaml':
#         print(path)



In [75]:
process_all_subdirectories(base_dir)

../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_1/2021-01-21_13-43-40
../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_3/2021-01-21_13-43-40
../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_2/2021-01-21_13-43-40
../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet20_5-preresnet20_1/trial_1/2021-01-21_13-43-40
../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet20_5-preresnet20_1/trial_3/2021-01-21_13-43-40
../data/experiments/image_classification/pavel_baseline_distillation_soft_only/s

In [91]:
trial_dir = "../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet20_3-preresnet20_1/trial_2/2021-01-21_13-43-40/"
with open(os.path.join(trial_dir, ".hydra/config.yaml"), "rb") as f:
    config = yaml.full_load(f)
student_model = make_resnet(config['classifier']['depth']).cuda()

with open(os.path.join(trial_dir, "student.ckpt"), "rb") as f:
    student_model.load_state_dict(pickle.load(f))

probs, labels = compute_probs(student_model, clean_test_loader)

In [93]:
get_accuracy(probs, labels)

0.6047

<All keys matched successfully>

In [69]:
test_path = "../data/experiments/image_classification/mixup/cifar100_ts_soft_cross_ent_alpha_0.0_v0.0.10/preresnet56_3-preresnet56_1/trial_1/2021-01-24_02-01-18/teacher_0.ckpt"
config_path = "../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet56_3-preresnet56_1/trial_1/2021-01-21_13-43-40/.hydra/config.yaml"

In [65]:
with open(config_path, "rb") as f:
    config = yaml.full_load(f)

In [68]:
config['classifier']['depth']

56

In [ ]:
for filename in 

In [70]:
os.listdir(test_path)

['teacher_test_logits.pkl',
 'hydra_config.txt',
 'student_train_metrics.csv',
 'student.ckpt',
 'events.out.tfevents.1611265439.mint',
 'teacher_train_logits.pkl',
 '.hydra',
 'student_train_logits.pkl',
 'image_classification.log',
 'student_test_logits.pkl',
 'teacher_ensemble.ckpt']

In [55]:
filepath = os.path.join(base_dir,
                        "preresnet110_1-preresnet110_1/trial_1/2021-01-21_13-43-40/.hydra")

In [13]:
base_dir = "../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/"
filepath = "preresnet110_1-preresnet110_1/trial_1/2021-01-21_13-43-40/teacher_test_logits.pkl"
filepath = os.path.join(base_dir, filepath)

with open(filepath, "rb") as f:
    logits = pickle.load(f)

In [15]:
logits.shape

torch.Size([10000, 100])

In [25]:
from collections import namedtuple

In [18]:
filepath = "../data/experiments/image_classification/pavel_baseline_distillation_soft_only/synth_aug_0.0_cifar100_ts_soft_cross_ent_v0.0.10/preresnet20_1-preresnet20_1/trial_1/2021-01-21_13-43-40/hydra_config.txt"
with open(filepath, "rb") as f:
    config = yaml.full_load(f)